# Brief 4 : Prédiction du prix immobilier

## Imports

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

## Lecture du csv et création du df

In [2]:
df = pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## Nettoyage des NaN

In [3]:
df['LotFrontage'] = df['LotFrontage'].replace(np.nan, 'No Street connected')
df['Alley'] = df['Alley'].replace(np.nan, 'No Alley')
df['MasVnrType'] = df['MasVnrType'].replace(np.nan, 'No MasVnr')
df['MasVnrArea'] = df['MasVnrArea'].replace(np.nan, 'No MasVnr')
df['BsmtQual'] = df['BsmtQual'].replace(np.nan, 'No Bsmt')
df['BsmtCond'] = df['BsmtCond'].replace(np.nan, 'No Bsmt')
df['BsmtExposure'] = df['BsmtExposure'].replace(np.nan, 'No Bsmt')
df['BsmtFinType1'] = df['BsmtFinType1'].replace(np.nan, 'No Bsmt')
df['BsmtFinType2'] = df['BsmtFinType2'].replace(np.nan, 'No 2ndBsmt')
df['FireplaceQu'] = df['FireplaceQu'].replace(np.nan, 'No Fireplace')
df['GarageType'] = df['GarageType'].replace(np.nan, 'No Garage')
df['GarageYrBlt'] = df['GarageYrBlt'].replace(np.nan, 'No Garage')
df['GarageFinish'] = df['GarageFinish'].replace(np.nan, 'No Garage')
df['GarageQual'] = df['GarageQual'].replace(np.nan, 'No Garage')
df['GarageCond'] = df['GarageCond'].replace(np.nan, 'No Garage')
df['PoolQC'] = df['PoolQC'].replace(np.nan, 'No Pool')
df['Fence'] = df['Fence'].replace(np.nan, 'No Fence')
df['MiscFeature'] = df['MiscFeature'].replace(np.nan, 'No Feature')
df = df.dropna(subset=['Electrical'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             1459 non-null   int64 
 1   MSSubClass     1459 non-null   int64 
 2   MSZoning       1459 non-null   object
 3   LotFrontage    1459 non-null   object
 4   LotArea        1459 non-null   int64 
 5   Street         1459 non-null   object
 6   Alley          1459 non-null   object
 7   LotShape       1459 non-null   object
 8   LandContour    1459 non-null   object
 9   Utilities      1459 non-null   object
 10  LotConfig      1459 non-null   object
 11  LandSlope      1459 non-null   object
 12  Neighborhood   1459 non-null   object
 13  Condition1     1459 non-null   object
 14  Condition2     1459 non-null   object
 15  BldgType       1459 non-null   object
 16  HouseStyle     1459 non-null   object
 17  OverallQual    1459 non-null   int64 
 18  OverallCond    1459 non-null

In [4]:
# Options d'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.DataFrame(df.loc[0:1459])

## Analyse exploratoire

In [5]:
df.describe()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,730.054832,56.881426,10517.363948,6.100069,5.575737,1971.244003,1984.850583,443.943797,46.581220,567.366004,1057.891021,1162.906785,346.791638,5.848526,1515.546950,0.425634,0.057574,1.564770,0.382454,2.866347,1.046607,6.517478,0.613434,1.766964,473.030158,94.240576,46.692255,21.969157,3.411926,15.071282,2.760795,43.518849,6.322824,2007.815627,180930.394791
std,421.411181,42.310746,9984.666267,1.383171,1.113079,30.199555,20.644343,456.106417,161.369977,441.992392,438.500975,386.572089,436.610637,48.639512,525.650927,0.518969,0.238830,0.550987,0.502798,0.816050,0.220410,1.625902,0.644687,0.747546,213.869604,125.381679,66.267472,61.137400,29.327247,55.775138,40.191018,496.291826,2.704331,1.328542,79468.964025
min,1.000000,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.500000,20.000000,7549.000000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,223.000000,796.000000,882.000000,0.000000,0.000000,1129.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1.000000,333.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129950.000000
50%,730.000000,50.000000,9477.000000,6.000000,5.000000,1973.000000,1994.000000,384.000000,0.000000,479.000000,992.000000,1088.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1094.500000,70.000000,11603.000000,7.000000,6.000000,2000.000000,2004.000000,712.500000,0.000000,808.000000,1298.500000,1391.500000,728.000000,0.000000,1777.500000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [6]:
# Moyennes des colonnes
df.mean()

Id                  730.054832
MSSubClass           56.881426
LotArea           10517.363948
OverallQual           6.100069
OverallCond           5.575737
YearBuilt          1971.244003
YearRemodAdd       1984.850583
BsmtFinSF1          443.943797
BsmtFinSF2           46.581220
BsmtUnfSF           567.366004
TotalBsmtSF        1057.891021
1stFlrSF           1162.906785
2ndFlrSF            346.791638
LowQualFinSF          5.848526
GrLivArea          1515.546950
BsmtFullBath          0.425634
BsmtHalfBath          0.057574
FullBath              1.564770
HalfBath              0.382454
BedroomAbvGr          2.866347
KitchenAbvGr          1.046607
TotRmsAbvGrd          6.517478
Fireplaces            0.613434
GarageCars            1.766964
GarageArea          473.030158
WoodDeckSF           94.240576
OpenPorchSF          46.692255
EnclosedPorch        21.969157
3SsnPorch             3.411926
ScreenPorch          15.071282
PoolArea              2.760795
MiscVal              43.518849
MoSold  

In [7]:
# Ecart-type des colonnes
df.std()

Id                 421.411181
MSSubClass          42.310746
LotArea           9984.666267
OverallQual          1.383171
OverallCond          1.113079
YearBuilt           30.199555
YearRemodAdd        20.644343
BsmtFinSF1         456.106417
BsmtFinSF2         161.369977
BsmtUnfSF          441.992392
TotalBsmtSF        438.500975
1stFlrSF           386.572089
2ndFlrSF           436.610637
LowQualFinSF        48.639512
GrLivArea          525.650927
BsmtFullBath         0.518969
BsmtHalfBath         0.238830
FullBath             0.550987
HalfBath             0.502798
BedroomAbvGr         0.816050
KitchenAbvGr         0.220410
TotRmsAbvGrd         1.625902
Fireplaces           0.644687
GarageCars           0.747546
GarageArea         213.869604
WoodDeckSF         125.381679
OpenPorchSF         66.267472
EnclosedPorch       61.137400
3SsnPorch           29.327247
ScreenPorch         55.775138
PoolArea            40.191018
MiscVal            496.291826
MoSold               2.704331
YrSold    

In [8]:
# Médianes des colonnes
df.median()

Id                  730.0
MSSubClass           50.0
LotArea            9477.0
OverallQual           6.0
OverallCond           5.0
YearBuilt          1973.0
YearRemodAdd       1994.0
BsmtFinSF1          384.0
BsmtFinSF2            0.0
BsmtUnfSF           479.0
TotalBsmtSF         992.0
1stFlrSF           1088.0
2ndFlrSF              0.0
LowQualFinSF          0.0
GrLivArea          1464.0
BsmtFullBath          0.0
BsmtHalfBath          0.0
FullBath              2.0
HalfBath              0.0
BedroomAbvGr          3.0
KitchenAbvGr          1.0
TotRmsAbvGrd          6.0
Fireplaces            1.0
GarageCars            2.0
GarageArea          480.0
WoodDeckSF            0.0
OpenPorchSF          25.0
EnclosedPorch         0.0
3SsnPorch             0.0
ScreenPorch           0.0
PoolArea              0.0
MiscVal               0.0
MoSold                6.0
YrSold             2008.0
SalePrice        163000.0
dtype: float64

## Suppression des colonnes inutiles

In [9]:
item_drop = ['MSSubClass','LotFrontage','Street','Alley','LandContour','Utilities','LotConfig','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior2nd','MasVnrType','MasVnrArea','ExterCond','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','1stFlrSF','2ndFlrSF','FullBath','HalfBath','KitchenAbvGr','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch','PoolArea','MoSold','YrSold','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','Electrical','LowQualFinSF','BsmtFullBath','BsmtHalfBath','FireplaceQu','GarageYrBlt','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','MiscVal','KitchenQual','3SsnPorch','LandSlope','OverallQual','OverallCond','YearRemodAdd','Exterior1st','ExterQual','Fireplaces','GarageCars','GarageType']

In [10]:
df = df.drop(columns=item_drop)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1459
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             1459 non-null   int64 
 1   MSZoning       1459 non-null   object
 2   LotArea        1459 non-null   int64 
 3   LotShape       1459 non-null   object
 4   Neighborhood   1459 non-null   object
 5   YearBuilt      1459 non-null   int64 
 6   Foundation     1459 non-null   object
 7   TotalBsmtSF    1459 non-null   int64 
 8   CentralAir     1459 non-null   object
 9   GrLivArea      1459 non-null   int64 
 10  BedroomAbvGr   1459 non-null   int64 
 11  TotRmsAbvGrd   1459 non-null   int64 
 12  Functional     1459 non-null   object
 13  GarageArea     1459 non-null   int64 
 14  SaleType       1459 non-null   object
 15  SaleCondition  1459 non-null   object
 16  SalePrice      1459 non-null   int64 
dtypes: int64(9), object(8)
memory usage: 205.2+ KB


In [ ]:
# df = prepare_train_data()
df_utilit = df[['SaleCondition', 'SalePrice']].groupby('SaleCondition').count()
df_utilit

In [ ]:
sns.pairplot(df)

In [ ]:
# corrmat = df.corr(df['SalePrice'])
# # f, ax = plt.subplots(figsize=(12, 6))
# # sns.heatmap(corrmat, square=True, annot=True);
# corrmat

corr_matrix=df.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

In [ ]:
results = smf.ols('SalePrice~YearBuilt+TotalBsmtSF+GrLivArea+GarageArea', data=df).fit()

# Inspect the results
print(results.summary())

In [ ]:
# X=pd.DataFrame(np.c_[df['YearBuilt'],df['TotalBsmtSF'],df['GrLivArea'],df['GarageArea']], columns = ['YearBuilt','TotalBsmtSf','GrLivArea','GarageArea'])
# Y = df['SalePrice']


# print(X_train.shape)
# print(X_test.shape)
# print(Y_train.shape)
# print(Y_test.shape)
# def create_list_numeric_columns(df):
#     return 

col_importante = [column for column in df.columns if df[column].dtypes == "int64"]

label = 'SalePrice'
# Separate features (columns 1 [house_age] to the last but one) and labels (the last column)
X, y = df[[col for col in df.columns if col != label]].values, df[label].values

# Split data 70%-30% into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training Set: %d, rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

In [ ]:


label = 'SalePrice'

X, y = df[[col for col in col_importante if col != label]].values, df[label].values

reg = LinearRegression().fit(X, y) 
reg.score(X, y)

reg.coef_

reg.intercept_

# >>> reg.predict(np.array([[3, 5]]))
# array([16.])

In [ ]:
reg.coef_

In [ ]:
reg.intercept_